In [66]:
#pip install psycopg

In [2]:
import sqlite3
import numpy as np
import pandas as pd
import psycopg2

In [3]:
conn = sqlite3.connect('PRA1.db')

In [4]:
cursor = conn.cursor()

In [5]:
with open('BBDD_ERP_data.sql', 'r') as s:
    script = s.read()
    cursor.executescript(script)

IntegrityError: UNIQUE constraint failed: tb_company.co_code

In [6]:
conn.commit()

In [7]:
#VER TABLAS DE LA BD

q=""" 
 SELECT * FROM sqlite_master WHERE type = "table";
 """
df = pd.read_sql(q, conn)
df

,type,name,tbl_name,rootpage,sql
0,table,tb_company,tb_company,2,CREATE TABLE tb_company\n (\n ...
1,table,tb_customer,tb_customer,4,CREATE TABLE tb_customer\n (\n ...
2,table,tb_site,tb_site,6,CREATE TABLE tb_site\n (\n c...
3,table,tb_iva,tb_iva,12,CREATE TABLE tb_iva\n (\n co...
4,table,tb_invoice,tb_invoice,8,CREATE TABLE tb_invoice\n (\n ...
5,table,tb_lines,tb_lines,10,CREATE TABLE tb_lines\n (\n ...


* a) Proporcionar los nombres de los clientes que empiezan por AGR.  
    Queremos obtener el número de cliente, nombre y NIF.  
    Los datos deben aparecer ordenados por número de cliente en orden descendiente.


In [40]:
q=""" 
SELECT cust_no, cust_name, cust_cif
FROM tb_customer
WHERE cust_name LIKE 'AGR%'
ORDER BY cust_no DESC
"""
df = pd.read_sql(q, conn)
df

,cust_no,cust_name,cust_cif
0,C0016,AGR SANT JOAN,B81232171
1,C0007,AGRIMARCOS,B98766562
2,C0006,AGR AGRI,B78788999
3,C0002,AGRICULTURA VIVES,A66666666


* b) Proporcionar las facturas de clientes con dirección en la ciudad de Granada.  
    Queremos obtener el número de cliente, nombre de cliente, dirección, ciudad y número de factura.  
    La consulta debe estar ordenada por número de cliente ascendientemente y número de factura descendentemente.

In [12]:
q=""" 
SELECT i.cust_no, c.cust_name, s.site_address, s.site_city, i.invoice_no
FROM tb_invoice as i
INNER JOIN tb_customer as c ON i.cust_no=c.cust_no
INNER JOIN tb_site as s ON c.cust_no=s.cust_no
WHERE UPPER(s.site_city) LIKE "GRANADA"
ORDER BY i.invoice_no ASC
"""
df = pd.read_sql(q, conn)
df.pivot_table(index=['cust_no', 'cust_name','site_address', 'site_city', 'invoice_no'])

Empty DataFrame
Columns: []
Index: [(C0010, VIAJES MUNDO, Av. Andalucia, 190, Granada, F0000C001000001), (C0010, VIAJES MUNDO, Av. Andalucia, 190, Granada, F0000C001000002), (C0010, VIAJES MUNDO, Av. Andalucia, 190, Granada, F0000C001000003), (C0010, VIAJES MUNDO, Av. Andalucia, 190, Granada, F0000C001000004), (C0010, VIAJES MUNDO, Av. Andalucia, 190, Granada, F0000C001000005), (C0010, VIAJES MUNDO, Av. Andalucia, 190, Granada, F0000C001000006), (C0010, VIAJES MUNDO, Av. Andalucia, 190, Granada, F0000C001000007), (C0010, VIAJES MUNDO, Av. Andalucia, 190, Granada, F0000C001000008), (C0010, VIAJES MUNDO, Av. Andalucia, 190, Granada, F0000C001000009), (C0010, VIAJES MUNDO, Av. Andalucia, 190, Granada, F0000C001000010), (C0010, VIAJES MUNDO, Av. Andalucia, 190, Granada, F0000C001000011), (C0010, VIAJES MUNDO, Av. Andalucia, 190, Granada, F0000C001000012), (C0010, VIAJES MUNDO, Av. Andalucia, 190, Granada, F0000C001000013), (C0010, VIAJES MUNDO, Av. Andalucia, 190, Granada, F0000C001000014), (C0010, VIAJES MUNDO, Av. Andalucia, 190, Granada, F0000C001000015), (C0010, VIAJES MUNDO, Av. Andalucia, 190, Granada, F0000C001000016), (C0010, VIAJES MUNDO, Av. Andalucia, 190, Granada, F0000C001000017), (C0010, VIAJES MUNDO, Av. Andalucia, 190, Granada, F0000C001000018), (C0010, VIAJES MUNDO, Av. Andalucia, 190, Granada, F0000C001000019), (C0010, VIAJES MUNDO, Av. Andalucia, 190, Granada, F0000C001000020), (C0010, VIAJES MUNDO, Av. Andalucia, 190, Granada, F0000C001000021), (C0010, VIAJES MUNDO, Av. Andalucia, 190, Granada, F0000C001000022), (C0010, VIAJES MUNDO, Av. Andalucia, 190, Granada, F0000C001000023), (C0010, VIAJES MUNDO, Av. Andalucia, 190, Granada, F0000C001000024)]

* c) Proporcionar la/s compañía/s con mayor número de facturas.  
    Queremos obtener el código de compañía, el nombre de la compañía y el número de facturas.  
    En caso de haber más de una compañía, los resultados deben aparecer ordenados por nombre de compañía.

In [92]:
q=""" 
SELECT c.co_code, c.co_name, invoice_no
FROM tb_company as c
INNER JOIN tb_invoice as i ON i.co_code=c.co_code


/**
GROUP BY i.co.code
ORDER BY co.name DESC, COUNT(i.invoice_no) DESC
*/
"""

df = pd.read_sql(q, conn)

df.groupby(["co_code","co_name"]).agg('size').sort_values(ascending=False)

co_code  co_name              
AB1      PGD CENTRAL INC          109
BC4      PGD ESTE INC             108
AA2      PGD SUR INC              102
XX6      PGD INTERNACIONAL INC     64
BB5      PGD OESTE INC             48
AX3      PGD NORTE INC             32
dtype: int64

* d) Proporcionar la lista de facturas que contienen más de 18 líneas de factura.  
    Queremos ver el número de factura, el cliente, el nombre de la compañía y el número de líneas en total.     
    La información debe estar ordenada por número de líneas de forma descendente y compañía.

In [42]:

q=""" 
SELECT i.invoice_no, cu.cust_name, co.co_name, l.line_num

FROM tb_lines as l

INNER JOIN tb_invoice as i ON l.invoice_id=i.invoice_id
INNER JOIN tb_customer as cu ON cu.cust_no=i.cust_no
INNER JOIN tb_company as co ON i.co_code=co.co_code

WHERE line_num>18
GROUP BY i.invoice_no

ORDER BY l.line_num DESC, co.co_name ASC
"""
df = pd.read_sql(q, conn)

df

,invoice_no,cust_name,co_name,line_num
0,F0000C001300066,GESTORIA LUIS,PGD SUR INC,23
1,F0000C000100105,PIENSOS MARTIN,PGD CENTRAL INC,19
2,F0000C000100112,PIENSOS MARTIN,PGD CENTRAL INC,19
3,F0000C000100113,PIENSOS MARTIN,PGD CENTRAL INC,19
4,F0000C000100114,PIENSOS MARTIN,PGD CENTRAL INC,19
...,...,...,...,...
116,F0000C001400096,CONSULTORES ORC,PGD SUR INC,19
117,F0000C001400097,CONSULTORES ORC,PGD SUR INC,19
118,F0000C001400099,CONSULTORES ORC,PGD SUR INC,19
119,F0000C001400100,CONSULTORES ORC,PGD SUR INC,19


In [23]:
q="""
SELECT invoice_no, cust_name, co_name, num_fac
FROM (
    SELECT invoice_no, tc.cust_name, tco.co_name, COUNT(1) num_fac
    FROM tb_invoice ti,
         tb_lines tl,
         tb_customer tc,
         tb_company tco         
    WHERE 1=1
    AND ti.invoice_id = tl.invoice_id
    AND tc.cust_no    = ti.cust_no
    and ti.co_code    = tco.co_code
    GROUP BY invoice_no, tc.cust_name, tco.co_name
) f
WHERE 1=1
AND num_fac >18
ORDER BY num_fac DESC, co_name
"""
df = pd.read_sql(q, conn)

df

,invoice_no,cust_name,co_name,num_fac
0,F0000C000200148,AGRICULTURA VIVES,PGD CENTRAL INC,24
1,F0000C000200157,AGRICULTURA VIVES,PGD CENTRAL INC,24
2,F0000C000200161,AGRICULTURA VIVES,PGD CENTRAL INC,24
3,F0000C000300187,CULTIVOS MARAVILLA,PGD CENTRAL INC,24
4,F0000C000400203,ASOCIADOS PEREZ,PGD CENTRAL INC,24
...,...,...,...,...
116,F0000C000500232,TECNICOS AVA,PGD NORTE INC,19
117,F0000C000500234,TECNICOS AVA,PGD NORTE INC,19
118,F0000C000600262,AGR AGRI,PGD OESTE INC,19
119,F0000C000600289,AGR AGRI,PGD OESTE INC,19


In [27]:
q=""" 
SELECT invoice_no, tc.cust_name, tco.co_name, COUNT(1) num_fac
    FROM tb_invoice ti,
         tb_lines tl,
         tb_customer tc,
         tb_company tco         
    WHERE 1=1
    AND ti.invoice_id = tl.invoice_id
    AND tc.cust_no    = ti.cust_no
    AND ti.co_code    = tco.co_code
    GROUP BY invoice_no, tc.cust_name, tco.co_name
    HAVING COUNT(1)>18
    ORDER BY num_fac DESC;
    """
df = pd.read_sql(q, conn)

df.pivot_table(index=['cust_name', 'co_name','invoice_no', 'num_fac'])

Empty DataFrame
Columns: []
Index: [(AGR AGRI, PGD OESTE INC, F0000C000600260, 21), (AGR AGRI, PGD OESTE INC, F0000C000600262, 19), (AGR AGRI, PGD OESTE INC, F0000C000600263, 23), (AGR AGRI, PGD OESTE INC, F0000C000600269, 21), (AGR AGRI, PGD OESTE INC, F0000C000600270, 21), (AGR AGRI, PGD OESTE INC, F0000C000600274, 21), (AGR AGRI, PGD OESTE INC, F0000C000600279, 21), (AGR AGRI, PGD OESTE INC, F0000C000600282, 22), (AGR AGRI, PGD OESTE INC, F0000C000600283, 20), (AGR AGRI, PGD OESTE INC, F0000C000600284, 23), (AGR AGRI, PGD OESTE INC, F0000C000600286, 22), (AGR AGRI, PGD OESTE INC, F0000C000600289, 19), (AGR AGRI, PGD OESTE INC, F0000C000600290, 21), (AGRICULTURA VIVES, PGD CENTRAL INC, F0000C000200138, 21), (AGRICULTURA VIVES, PGD CENTRAL INC, F0000C000200144, 19), (AGRICULTURA VIVES, PGD CENTRAL INC, F0000C000200146, 20), (AGRICULTURA VIVES, PGD CENTRAL INC, F0000C000200148, 24), (AGRICULTURA VIVES, PGD CENTRAL INC, F0000C000200150, 23), (AGRICULTURA VIVES, PGD CENTRAL INC, F0000C000200152, 22), (AGRICULTURA VIVES, PGD CENTRAL INC, F0000C000200155, 21), (AGRICULTURA VIVES, PGD CENTRAL INC, F0000C000200157, 24), (AGRICULTURA VIVES, PGD CENTRAL INC, F0000C000200159, 22), (AGRICULTURA VIVES, PGD CENTRAL INC, F0000C000200161, 24), (AGRICULTURA VIVES, PGD CENTRAL INC, F0000C000200181, 20), (AGRICULTURA VIVES, PGD CENTRAL INC, F0000C000200182, 20), (AGRIMARCOS, PGD ESTE INC, F0000C000700292, 22), (AGRIMARCOS, PGD ESTE INC, F0000C000700293, 22), (AGRIMARCOS, PGD ESTE INC, F0000C000700299, 19), (AGRIMARCOS, PGD ESTE INC, F0000C000700307, 24), (AGRIMARCOS, PGD ESTE INC, F0000C000700308, 21), (AGRIMARCOS, PGD ESTE INC, F0000C000700312, 20), (APR T, PGD INTERNACIONAL INC, F0000C001500408, 23), (APR T, PGD INTERNACIONAL INC, F0000C001500411, 24), (APR T, PGD INTERNACIONAL INC, F0000C001500413, 23), (APR T, PGD INTERNACIONAL INC, F0000C001500415, 24), (APR T, PGD INTERNACIONAL INC, F0000C001500418, 22), (APR T, PGD INTERNACIONAL INC, F0000C001500420, 19), (APR T, PGD INTERNACIONAL INC, F0000C001500422, 24), (APR T, PGD INTERNACIONAL INC, F0000C001500438, 24), (APR T, PGD INTERNACIONAL INC, F0000C001500444, 21), (APR T, PGD INTERNACIONAL INC, F0000C001500448, 19), (APR T, PGD INTERNACIONAL INC, F0000C001500450, 23), (APR T, PGD INTERNACIONAL INC, F0000C001500452, 21), (ASOCIADOS PEREZ, PGD CENTRAL INC, F0000C000400200, 22), (ASOCIADOS PEREZ, PGD CENTRAL INC, F0000C000400203, 24), (ASOCIADOS PEREZ, PGD CENTRAL INC, F0000C000400210, 21), (CONSULTORES ORC, PGD SUR INC, F0000C001400093, 23), (CONSULTORES ORC, PGD SUR INC, F0000C001400095, 21), (CONSULTORES ORC, PGD SUR INC, F0000C001400096, 20), (CONSULTORES ORC, PGD SUR INC, F0000C001400097, 20), (CONSULTORES ORC, PGD SUR INC, F0000C001400099, 20), (CONSULTORES ORC, PGD SUR INC, F0000C001400100, 22), (CONSULTORES ORC, PGD SUR INC, F0000C001400102, 23), (CULTIVANDO ALEGRIA, PGD ESTE INC, F0000C000800325, 20), (CULTIVANDO ALEGRIA, PGD ESTE INC, F0000C000800329, 24), (CULTIVANDO ALEGRIA, PGD ESTE INC, F0000C000800330, 21), (CULTIVANDO ALEGRIA, PGD ESTE INC, F0000C000800331, 22), (CULTIVANDO ALEGRIA, PGD ESTE INC, F0000C000800333, 23), (CULTIVANDO ALEGRIA, PGD ESTE INC, F0000C000800336, 24), (CULTIVANDO ALEGRIA, PGD ESTE INC, F0000C000800337, 21), (CULTIVANDO ALEGRIA, PGD ESTE INC, F0000C000800339, 20), (CULTIVANDO ALEGRIA, PGD ESTE INC, F0000C000800341, 22), (CULTIVANDO ALEGRIA, PGD ESTE INC, F0000C000800350, 24), (CULTIVANDO ALEGRIA, PGD ESTE INC, F0000C000800353, 21), (CULTIVANDO ALEGRIA, PGD ESTE INC, F0000C000800362, 24), (CULTIVANDO ALEGRIA, PGD ESTE INC, F0000C000800363, 19), (CULTIVANDO ALEGRIA, PGD ESTE INC, F0000C000800368, 22), (CULTIVANDO ALEGRIA, PGD ESTE INC, F0000C000800371, 24), (CULTIVANDO ALEGRIA, PGD ESTE INC, F0000C000800374, 22), (CULTIVANDO ALEGRIA, PGD ESTE INC, F0000C000800375, 24), (CULTIVANDO ALEGRIA, PGD ESTE INC, F0000C000800376, 20), (CULTIVANDO ALEGRIA, PGD ESTE INC, F0000C000800381, 19), (CULTIVOS MARAVILLA, PGD CENTRAL INC, F0000C000300187, 24), (CULTIVOS MARAVIL

* e) Proporcionar una lista con el importe de las facturas pendientes de cobrar,  
    es decir, aquellas que no hayan sido pagadas.       
    Queremos ver el código de compañía, nombre de la compañía, número de cliente, nombre de cliente, total IVA y total factura (tot_amount).        
    Los resultados deben aparecer ordenados por nombre de compañía ascendiente y código de cliente descendentemente

In [15]:
q=""" 
SELECT i.co_code, co.co_name, cu.cust_no, cu.cust_name, i.iva_amount, i.tot_amount
FROM tb_invoice as i
INNER JOIN tb_company as co ON i.co_code=co.co_code
INNER JOIN tb_customer as cu ON i.cust_no=cu.cust_no
WHERE payed="N"
ORDER BY co.co_name ASC, i.cust_no DESC
"""
df = pd.read_sql(q, conn)

df.pivot_table(index=['co_code', 'co_name','cust_no', 'cust_name'], aggfunc= 'sum')

iva_amount   
co_code co_name               cust_no cust_name                        
AA2     PGD SUR INC           C0010   VIAJES MUNDO          43240.97  \
                              C0011   SEMILLAS MARIA        20000.25   
                              C0012   PIENSOS JAIME         19589.73   
                              C0013   GESTORIA LUIS         51416.31   
                              C0014   CONSULTORES ORC       22373.75   
AB1     PGD CENTRAL INC       C0001   PIENSOS MARTIN        27729.28   
                              C0002   AGRICULTURA VIVES     44762.67   
                              C0003   CULTIVOS MARAVILLA    12789.14   
                              C0004   ASOCIADOS PEREZ        9263.33   
AX3     PGD NORTE INC         C0005   TECNICOS AVA          42595.05   
BB5     PGD OESTE INC         C0005   TECNICOS AVA          23215.68   
                              C0006   AGR AGRI              62760.73   
BC4     PGD ESTE INC          C0007   AGRIMARCOS            33545.73   
                              C0008   CULTIVANDO ALEGRIA    76471.36   
                              C0009   MARCOS LIMPIEZA       11199.55   
XX6     PGD INTERNACIONAL INC C0015   APR T                 78873.81   

                                                          tot_amount  
co_code co_name               cust_no cust_name                       
AA2     PGD SUR INC           C0010   VIAJES MUNDO         249150.49  
                              C0011   SEMILLAS MARIA       115239.55  
                              C0012   PIENSOS JAIME        112873.94  
                              C0013   GESTORIA LUIS        296255.66  
                              C0014   CONSULTORES ORC      128915.43  
AB1     PGD CENTRAL INC       C0001   PIENSOS MARTIN       254510.80  
                              C0002   AGRICULTURA VIVES    403042.85  
                              C0003   CULTIVOS MARAVILLA   117387.48  
                              C0004   ASOCIADOS PEREZ       81601.96  
AX3     PGD NORTE INC         C0005   TECNICOS AVA         245428.49  
BB5     PGD OESTE INC         C0005   TECNICOS AVA         133766.59  
                              C0006   AGR AGRI             361621.32  
BC4     PGD ESTE INC          C0007   AGRIMARCOS           193287.21  
                              C0008   CULTIVANDO ALEGRIA   440620.32  
                              C0009   MARCOS LIMPIEZA       64530.69  
XX6     PGD INTERNACIONAL INC C0015   APR T                614286.62

In [34]:
q=""" 
SELECT i.co_code, co.co_name, cu.cust_no, cu.cust_name, SUM(i.iva_amount) as suma_IVA, SUM(i.tot_amount) as suma_TOTAL
FROM tb_invoice as i
INNER JOIN tb_company as co ON i.co_code=co.co_code
INNER JOIN tb_customer as cu ON i.cust_no=cu.cust_no
WHERE payed="N"

ORDER BY co.co_name ASC, i.cust_no DESC
"""
df = pd.read_sql(q, conn)
df

,co_code,co_name,cust_no,cust_name,suma_IVA,suma_TOTAL
0,AB1,PGD CENTRAL INC,C0004,ASOCIADOS PEREZ,579827.34,3812519.4
